In [1]:
import pandas as pd
import re
import requests

In [2]:
from io import StringIO

In [3]:
#Life expectancy at birth -> LEB
#Life Expectancy Index

#Expected years of schooling -> EYS                                    
#Mean years of schooling -> MYS
#Education Index

#Gross national income -> GNI
#GNI Index

#(Life Expectancy Index, Education Index, GNI Index) -> Human Development Index -> HDI

#Gross Domestic Product -> GDP

In [4]:
country_mapping = {
    "Hong Kong, China (SAR)": "Hong Kong",
    "UK": "United Kingdom",
    "UAE": "United Arab Emirates",
    "S. Korea": "South Korea",
    "Korea (Republic of)": "South Korea",
    "USA": "United States",
    "Czech Republic (Czechia)": "Czechia",
    "Türkiye": "Turkey",
    "Saint Kitts & Nevis": "Saint Kitts and Nevis",
    "Brunei Darussalam" : "Brunei",
    "Russian Federation": "Russia",
    "Iran (Islamic Republic of)": "Iran",
    "Moldova (Republic of)": "Moldova",
    "Viet Nam": "Vietnam",
    "Palestine, State of": "Palestine",
    "State of Palestine": "Palestine",
    "Bolivia (Plurinational State of)": "Bolivia",
    "Micronesia (Federated States of)": "Micronesia",
    "Lao People's Democratic Republic": "Laos",
    "Sao Tome & Principe" : "Sao Tome and Principe",
    "Eswatini (Kingdom of)" : "Eswatini",
    "Côte d'Ivoire": "Ivory Coast",
    "Tanzania (United Republic of)": "Tanzania",
    "Congo (Democratic Republic of the)": "DR Congo",
    "DRC": "DR Congo",
    "CAR": "Central African Republic"
}

In [5]:
def normalize_country(dataframe):
    dataframe['Country'] = dataframe['Country'].replace(country_mapping)
    return dataframe

def transform_gdp_value(gdp_value):
    return int(gdp_value.replace("$", "").replace(",", ""))

def transform_gdp_percentage(gdp_value):
    return float(gdp_value.replace("%", ""))

def transform_float_to_int(value):
    return int(value)

In [8]:
dataFrameHDI = pd.read_excel("dataset/HDR23-24_Statistical_HDI_Covenant.xlsx")

In [9]:
dataFrameHDI.columns

Index(['HDI rank', 'Country', 'Human Development Index (HDI) 2022',
       'Life expectancy at birth 2022', 'Expected years of schooling 2022',
       'Mean years of schooling 2022',
       'Gross national income (GNI) per capita 2022',
       'GNI per capita rank minus HDI rank 2022', 'HDI rank 2021'],
      dtype='object')

In [10]:
dataFrameHDI = dataFrameHDI.drop(columns=['HDI rank 2021', 'GNI per capita rank minus HDI rank 2022'])
dataFrameHDI.dropna(inplace=True)
dataFrameHDI.columns = ['HDI Rank(2022)', 'Country', 'HDI(2022)','LEB(2022)', 'EYS(2022)', 'MYS(2022)','GNI(Per Capita,2022)']
dataFrameHDI = dataFrameHDI[['Country','HDI Rank(2022)', 'HDI(2022)','LEB(2022)', 'EYS(2022)', 'MYS(2022)','GNI(Per Capita,2022)']]

In [11]:
dataFrameHDI = normalize_country(dataFrameHDI)

In [12]:
dataFrameHDI

,Country,HDI Rank(2022),HDI(2022),LEB(2022),EYS(2022),MYS(2022),"GNI(Per Capita,2022)"
0,Switzerland,1,0.967,84.255,16.583731,13.904066,69432.786690
1,Norway,2,0.966,83.393,18.638460,13.062343,69189.761650
2,Iceland,3,0.959,82.815,19.106730,13.767170,54688.379210
3,Hong Kong,4,0.956,84.315,17.849590,12.347770,62485.505160
4,Denmark,5,0.952,81.882,18.774031,12.960490,62018.956940
...,...,...,...,...,...,...,...
188,Chad,189,0.394,52.997,8.185527,2.282753,1388.897502
189,Niger,189,0.394,62.080,7.186131,1.341352,1283.309235
190,Central African Republic,191,0.387,54.477,7.285887,3.953164,869.112370
191,South Sudan,192,0.381,55.567,5.634770,5.726140,690.660758


In [13]:
pathGDP = 'https://www.worldometers.info/gdp/gdp-by-country/'

In [16]:
requestGDP = requests.get(pathGDP)

In [17]:
dataFrameGDP = pd.read_html(StringIO(requestGDP.text))

In [18]:
dataFrameGDP = dataFrameGDP[0]
dataFrameGDP.columns

Index(['#', 'Country', 'GDP  (nominal, 2022)', 'GDP  (abbrev.)', 'GDP growth',
       'Population  (2022)', 'GDP per capita', 'Share of  World GDP'],
      dtype='object')

In [19]:
dataFrameGDP = dataFrameGDP.drop(columns=['#','GDP  (abbrev.)', ])
dataFrameGDP.dropna(inplace=True)
dataFrameGDP.columns = ['Country', 'GDP(Nominal,2022)', 'GDP(Growth)', 'Population(2022)', 'GDP(Per Capita)', 'GDP(Share of World)']
dataFrameGDP = dataFrameGDP[['Country', 'Population(2022)', 'GDP(Nominal,2022)', 'GDP(Growth)', 'GDP(Per Capita)', 'GDP(Share of World)']]

In [20]:
dataFrameGDP['GDP(Nominal,2022)'] = dataFrameGDP['GDP(Nominal,2022)'].apply(transform_gdp_value)
dataFrameGDP['GDP(Growth)'] = dataFrameGDP['GDP(Growth)'].apply(transform_gdp_percentage)
dataFrameGDP['GDP(Per Capita)'] = dataFrameGDP['GDP(Per Capita)'].apply(transform_gdp_value)
dataFrameGDP['GDP(Share of World)'] = dataFrameGDP['GDP(Share of World)'].apply(transform_gdp_percentage)

In [21]:
dataFrameGDP = normalize_country(dataFrameGDP)

In [22]:
dataFrameGDP

,Country,Population(2022),"GDP(Nominal,2022)",GDP(Growth),GDP(Per Capita),GDP(Share of World)
0,United States,338289857,25462700000000,2.06,75269,25.32
1,China,1425887337,17963200000000,2.99,12598,17.86
2,Japan,123951692,4231140000000,1.03,34135,4.21
3,Germany,83369843,4072190000000,1.79,48845,4.05
4,India,1417173173,3385090000000,7.00,2389,3.37
...,...,...,...,...,...,...
172,Sao Tome and Principe,227380,546680342,0.93,2404,0.00
173,Micronesia,539013,427094119,-0.62,792,0.00
174,Marshall Islands,41569,279667900,1.50,6728,0.00
175,Kiribati,131232,223352943,1.56,1702,0.00


In [23]:
pathCovid = 'https://www.worldometers.info/coronavirus/'

In [24]:
requestCovid = requests.get(pathCovid)

In [25]:
sourceCovid = re.sub(r'<.*?>', lambda g: g.group(0).upper(), requestCovid.text)

In [26]:
dataFrameCovid = pd.read_html(StringIO(sourceCovid))

In [27]:
dataFrameCovid[1].head(15)

,#,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,"Serious, Critical",...,Total Tests,Tests/ 1M pop,Population,Continent,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,NaN,Asia,221500265,764.0,1553662.0,34.0,205673125.0,91.0,14273478.0,14730.0,...,NaN,NaN,NaN,Asia,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,North America,131889132,1151.0,1695941.0,7.0,127664779.0,10459.0,2528412.0,6095.0,...,NaN,NaN,NaN,North America,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Europe,253406198,165.0,2101824.0,16.0,248753630.0,104.0,2550744.0,4453.0,...,NaN,NaN,NaN,Europe,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,South America,70200879,1215.0,1367332.0,NaN,66683585.0,NaN,2149962.0,8953.0,...,NaN,NaN,NaN,South America,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Oceania,14895771,664.0,33015.0,NaN,14752388.0,4299.0,110368.0,31.0,...,NaN,NaN,NaN,Australia/Oceania,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Africa,12860924,NaN,258892.0,NaN,12090808.0,NaN,511224.0,529.0,...,NaN,NaN,NaN,Africa,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,721,NaN,15.0,NaN,706.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,World,704753890,3959.0,7010681.0,57.0,675619021.0,14953.0,22124188.0,34791.0,...,NaN,NaN,NaN,All,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,China,503302,NaN,5272.0,NaN,379053.0,NaN,118977.0,7557.0,...,1.600000e+08,110461.0,1.448471e+09,Asia,2878.0,274748.0,9.0,NaN,NaN,82.0
9,2.0,USA,111820082,1151.0,1219487.0,7.0,109814428.0,10109.0,786167.0,940.0,...,1.186852e+09,3544901.0,3.348053e+08,North America,3.0,275.0,0.0,3.00,0.02,2348.0


In [28]:
dataFrameCovid = dataFrameCovid[1]
dataFrameCovid.columns

Index(['#', 'Country, Other', 'Total Cases', 'New Cases', 'Total Deaths',
       'New Deaths', 'Total Recovered', 'New Recovered', 'Active Cases',
       'Serious, Critical', 'Tot Cases/ 1M pop', 'Deaths/ 1M pop',
       'Total Tests', 'Tests/  1M pop', 'Population', 'Continent',
       '1 Case every X ppl', '1 Death every X ppl', '1 Test every X ppl',
       'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop'],
      dtype='object')

In [29]:
dataFrameCovid.iloc[2].index

Index(['#', 'Country, Other', 'Total Cases', 'New Cases', 'Total Deaths',
       'New Deaths', 'Total Recovered', 'New Recovered', 'Active Cases',
       'Serious, Critical', 'Tot Cases/ 1M pop', 'Deaths/ 1M pop',
       'Total Tests', 'Tests/  1M pop', 'Population', 'Continent',
       '1 Case every X ppl', '1 Death every X ppl', '1 Test every X ppl',
       'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop'],
      dtype='object')

In [30]:
dataFrameCovid.drop(dataFrameCovid.tail(8).index, inplace = True)
dataFrameCovid.drop(dataFrameCovid.head(8).index, inplace = True)
dataFrameCovid = dataFrameCovid.reset_index(drop=True)
dataFrameCovid.shape

(231, 22)

In [31]:
dataFrameCovid.describe()

,#,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,"Serious, Critical",Tot Cases/ 1M pop,Deaths/ 1M pop,Total Tests,Tests/ 1M pop,Population,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
count,231.000000,2.310000e+02,9.000000,2.260000e+02,3.000000,1.830000e+02,7.000000,1.840000e+02,52.000000,229.000000,224.000000,2.130000e+02,2.130000e+02,2.290000e+02,229.000000,224.000000,213.000000,9.000000,3.000000,203.000000
mean,116.000000,3.050883e+06,439.888889,3.102071e+04,19.000000,3.063207e+06,2123.142857,3.324186e+04,200.596154,202950.253275,1277.348214,3.298829e+07,2.155978e+06,3.469404e+07,401.510917,14392.781250,10.685446,22.261111,1.406667,33323.278227
std,66.828138,1.002125e+07,499.992611,1.103436e+05,13.747727,1.024277e+07,3856.696143,1.654546e+05,1049.899346,206154.889634,1310.431844,1.198779e+08,3.647972e+06,1.386374e+08,4147.207187,57069.426357,26.734890,33.045346,2.247695,97173.888563
min,1.000000,9.000000e+00,3.000000,1.000000e+00,7.000000,2.000000e+00,17.000000,0.000000e+00,1.000000,16.000000,2.000000,7.850000e+03,5.093000e+03,7.990000e+02,1.000000,152.000000,0.000000,0.050000,0.020000,-168.580000
25%,58.500000,2.636600e+04,68.000000,2.055000e+02,11.500000,1.902150e+04,25.500000,2.875000e+01,4.000000,18885.000000,172.750000,3.478150e+05,1.981990e+05,4.454310e+05,3.000000,478.000000,0.000000,0.900000,0.110000,48.500000
50%,116.000000,2.068970e+05,73.000000,2.204500e+03,16.000000,1.870780e+05,36.000000,5.640000e+02,11.500000,128843.000000,846.500000,2.226216e+06,8.851190e+05,5.797805e+06,8.000000,1181.500000,1.000000,8.000000,0.200000,613.000000
75%,173.500000,1.348198e+06,696.000000,1.480600e+04,25.000000,1.503534e+06,2324.500000,6.105000e+03,26.500000,333985.000000,2092.250000,1.346373e+07,2.439107e+06,2.210284e+07,53.000000,5798.500000,5.000000,25.000000,2.100000,4075.500000
max,231.000000,1.118201e+08,1215.000000,1.219487e+06,34.000000,1.098144e+08,10109.000000,1.783377e+06,7557.000000,771655.000000,6595.000000,1.186852e+09,2.330212e+07,1.448471e+09,62616.000000,626161.000000,196.000000,92.000000,4.000000,612553.000000


In [32]:
dataFrameCovid              

,#,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,"Serious, Critical",...,Total Tests,Tests/ 1M pop,Population,Continent,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,1.0,China,503302,NaN,5272.0,NaN,379053.0,NaN,118977.0,7557.0,...,1.600000e+08,110461.0,1.448471e+09,Asia,2878.0,274748.0,9.0,NaN,NaN,82.0
1,2.0,USA,111820082,1151.0,1219487.0,7.0,109814428.0,10109.0,786167.0,940.0,...,1.186852e+09,3544901.0,3.348053e+08,North America,3.0,275.0,0.0,3.00,0.02,2348.0
2,3.0,India,45035393,68.0,533570.0,NaN,NaN,NaN,NaN,NaN,...,9.358795e+08,665334.0,1.406632e+09,Asia,31.0,2636.0,2.0,0.05,NaN,0.4
3,4.0,France,40138560,NaN,167642.0,NaN,39970918.0,NaN,0.0,NaN,...,2.714902e+08,4139547.0,6.558452e+07,Europe,2.0,391.0,0.0,NaN,NaN,NaN
4,5.0,Germany,38828995,73.0,183027.0,16.0,38240600.0,NaN,405368.0,NaN,...,1.223324e+08,1458359.0,8.388360e+07,Europe,2.0,458.0,1.0,0.90,0.20,4833.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,227.0,Diamond Princess,712,NaN,13.0,NaN,699.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,228.0,Tokelau,80,NaN,NaN,NaN,NaN,NaN,80.0,NaN,...,NaN,NaN,1.378000e+03,Australia/Oceania,17.0,NaN,NaN,NaN,NaN,58055.0
228,229.0,Vatican City,29,NaN,NaN,NaN,29.0,NaN,0.0,NaN,...,NaN,NaN,7.990000e+02,Europe,28.0,NaN,NaN,NaN,NaN,NaN
229,230.0,Western Sahara,10,NaN,1.0,NaN,9.0,NaN,0.0,NaN,...,NaN,NaN,6.261610e+05,Africa,62616.0,626161.0,NaN,NaN,NaN,NaN


In [33]:
dataFrameHDICovenant = dataFrameHDI
dataFrameHDICovenant = dataFrameHDICovenant.drop(columns=['LEB(2022)', 'EYS(2022)', 'MYS(2022)','GNI(Per Capita,2022)'])
dataFrameHDICovenant.dropna(inplace=True)

dataFrameHDICovenant = dataFrameHDICovenant[['Country', 'HDI Rank(2022)', 'HDI(2022)']]

In [34]:
dataFrameGDPCovenant = dataFrameGDP
dataFrameGDPCovenant = dataFrameGDPCovenant.drop(columns=['GDP(Growth)', 'GDP(Share of World)'])
dataFrameGDPCovenant.dropna(inplace=True)

In [42]:
dataFrameCovidCovenant = dataFrameCovid
dataFrameCovidCovenant = dataFrameCovidCovenant.drop(columns=['#',  'Total Cases', 'New Cases', 'Total Deaths',
       'New Deaths', 'Total Recovered', 'New Recovered', 'Active Cases',
       'Serious, Critical',  'Total Tests', 'Population', 'Continent', '1 Case every X ppl',
       '1 Death every X ppl', '1 Test every X ppl', 'New Cases/1M pop',
       'New Deaths/1M pop', 'Active Cases/1M pop'])
dataFrameCovidCovenant.dropna(inplace=True)

In [43]:
dataFrameCovidCovenant.columns = ['Country', 'TotCases1Mpop', 'Deaths1Mpop', 'Tests1Mpop']
dataFrameCovidCovenant['TotCases1Mpop'] = dataFrameCovidCovenant['TotCases1Mpop'].apply(transform_float_to_int)
dataFrameCovidCovenant['Deaths1Mpop'] = dataFrameCovidCovenant['Deaths1Mpop'].apply(transform_float_to_int)
dataFrameCovidCovenant['Tests1Mpop'] = dataFrameCovidCovenant['Tests1Mpop'].apply(transform_float_to_int)
dataFrameCovidCovenant = normalize_country(dataFrameCovidCovenant)

dataFrameCovidCovenant

,Country,TotCases1Mpop,Deaths1Mpop,Tests1Mpop
0,China,347,4,110461
1,United States,333985,3642,3544901
2,India,32016,379,665334
3,France,612013,2556,4139547
4,Germany,462891,2182,1458359
...,...,...,...,...
216,Nauru,494635,92,1881042
218,Anguilla,256336,788,3373736
219,Wallis and Futuna,323256,728,1867419
220,Saint Pierre Miquelon,599410,347,4410488


In [ ]:
gdp_hdi_Covenant = dataFrameHDICovenant.merge(dataFrameGDPCovenant, how='inner', on='Country')

In [ ]:
dfCovidEconomyHuman = gdp_hdi_Covenant.merge(dataFrameCovidCovenant, how='inner', on='Country')

In [ ]:
dfCovidEconomyHuman = dfCovidEconomyHuman[~dfCovidEconomyHuman['Population(2022)'].isna()]
dfCovidEconomyHuman = dfCovidEconomyHuman.reset_index(drop=True)
dfCovidEconomyHuman

In [ ]:
dfCovidEconomyHuman.to_pickle('dataframe/covidEconomyHuman.pkl')